In [ ]:
!pip install pandas openai requests

In [ ]:
!pip install tqdm

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import openai
import requests
from tqdm import tqdm
import time
import docx

In [39]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [59]:
input_file = "data/amazon.csv"
input_file_df = pd.read_csv(input_file)
display(input_file_df.head())

df = input_file_df[["id","reviews.text"]][:10]
df.head()

<ipython-input-59-9fd88c857a41>:2: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  input_file_df = pd.read_csv(input_file)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


,id,reviews.text
0,AVqkIhwDv8e3D1O-lebb,This product so far has not disappointed. My c...
1,AVqkIhwDv8e3D1O-lebb,great for beginner or experienced person. Boug...
2,AVqkIhwDv8e3D1O-lebb,Inexpensive tablet for him to use and learn on...
3,AVqkIhwDv8e3D1O-lebb,I've had my Fire HD 8 two weeks now and I love...
4,AVqkIhwDv8e3D1O-lebb,I bought this for my grand daughter when she c...


In [60]:
df

,id,reviews.text
0,AVqkIhwDv8e3D1O-lebb,This product so far has not disappointed. My c...
1,AVqkIhwDv8e3D1O-lebb,great for beginner or experienced person. Boug...
2,AVqkIhwDv8e3D1O-lebb,Inexpensive tablet for him to use and learn on...
3,AVqkIhwDv8e3D1O-lebb,I've had my Fire HD 8 two weeks now and I love...
4,AVqkIhwDv8e3D1O-lebb,I bought this for my grand daughter when she c...
5,AVqkIhwDv8e3D1O-lebb,This amazon fire 8 inch tablet is the perfect ...
6,AVqkIhwDv8e3D1O-lebb,"Great for e-reading on the go, nice and light ..."
7,AVqkIhwDv8e3D1O-lebb,"I gave this as a Christmas gift to my inlaws, ..."
8,AVqkIhwDv8e3D1O-lebb,Great as a device to read books. I like that i...
9,AVqkIhwDv8e3D1O-lebb,I love ordering books and reading them with th...


In [61]:
def analyze_review(review):
  retries = 3
  sentiment = None

  while retries >0:
    messages = [
            {"role": "system", "content": "You are an AI language model trained to analyze and detect the sentiment of product reviews."},
            {"role": "user", "content": f"Analyze the following product review and determine if the sentiment is: positive, negative or neutral. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL: {review}"}
    ]
    completion = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages=messages,
        # max_token=3,
        # n=1,
        # stop=None,
        # temperature=0
    )

    response_text = completion.choices[0].message.content
    print(response_text)
    if response_text in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
      sentiment = response_text
      break
    else:
      retries -= 1
      time.sleep(1)
  else:
    sentiment="neutral"
  retries=3

  time.sleep(21)
  return sentiment


sentiments = []
for review in tqdm(df["reviews.text"], desc="Processing reviews"):
  sentiment = analyze_review(review)
  sentiments.append(sentiment)

df["sentiment"] = sentiments



Processing reviews:   0%|          | 0/10 [00:00<?, ?it/s]

POSITIVE


Processing reviews:  10%|█         | 1/10 [00:23<03:30, 23.44s/it]

POSITIVE


Processing reviews:  20%|██        | 2/10 [00:44<02:58, 22.31s/it]

...

POSITIVE


Processing reviews:  90%|█████████ | 9/10 [03:18<00:21, 21.89s/it]

POSITIVE


Processing reviews: 100%|██████████| 10/10 [03:40<00:00, 22.09s/it]


KeyError: ignored

In [62]:


# Save the results to a new Excel file
output_file = "reviews_analyzed_full_sentiment.xlsx"
df.to_excel(output_file, index=False)


# Save the results to a new Word file
output_file = "reviews_analyzed_full_sentiment.docx"
doc = docx.Document()

# Add table with headers
table = doc.add_table(rows=1, cols=2)
header_cells = table.rows[0].cells
header_cells[0].text = 'Product_Review'
header_cells[1].text = 'Sentiment'

# Add table content
for index, row in df.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = str(row['reviews.text'])
    row_cells[1].text = row['sentiment']

doc.save(output_file)


In [63]:
df

,id,reviews.text,sentiment
0,AVqkIhwDv8e3D1O-lebb,This product so far has not disappointed. My c...,POSITIVE
1,AVqkIhwDv8e3D1O-lebb,great for beginner or experienced person. Boug...,POSITIVE
2,AVqkIhwDv8e3D1O-lebb,Inexpensive tablet for him to use and learn on...,POSITIVE
3,AVqkIhwDv8e3D1O-lebb,I've had my Fire HD 8 two weeks now and I love...,POSITIVE
4,AVqkIhwDv8e3D1O-lebb,I bought this for my grand daughter when she c...,POSITIVE
5,AVqkIhwDv8e3D1O-lebb,This amazon fire 8 inch tablet is the perfect ...,POSITIVE
6,AVqkIhwDv8e3D1O-lebb,"Great for e-reading on the go, nice and light ...",POSITIVE
7,AVqkIhwDv8e3D1O-lebb,"I gave this as a Christmas gift to my inlaws, ...",POSITIVE
8,AVqkIhwDv8e3D1O-lebb,Great as a device to read books. I like that i...,POSITIVE
9,AVqkIhwDv8e3D1O-lebb,I love ordering books and reading them with th...,POSITIVE
